In [1]:
import pandas as pd

In [2]:
def get_label_text(path):
    ''''
    A function that preprocesses the text and returns text with labels
    '''
    df = pd.read_csv(path)
    df.head()
    text = df['sentence'] 
    t = []
    for i in text:
        t.append(i.lower().split())
    label = df['tags']
    l=[]
    for i in label:
        l.append(i.split())
    return t,l

In [3]:
train_text,train_label =get_label_text("ner_train.csv")
val_text,val_label =get_label_text("ner_val.csv")
print("Length TRAIN\n Text:",len(train_text),"\t","Label:",len(train_label))
print("Length val\n Text:",len(val_text),"\t","Label:",len(val_label))

Length TRAIN
 Text: 38367 	 Label: 38367
Length val
 Text: 4796 	 Label: 4796


In [4]:
import torch
seed = 7777
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
import torch.nn as nn
import torch.optim as optim

In [5]:
tag = list(sorted(set(w for s in train_label for w in s)))

In [6]:
print(tag)

['B-art', 'B-eve', 'B-geo', 'B-gpe', 'B-nat', 'B-org', 'B-per', 'B-tim', 'I-art', 'I-eve', 'I-geo', 'I-gpe', 'I-nat', 'I-org', 'I-per', 'I-tim', 'O']


In [7]:
tag_to_ix = {t: i for i, t in enumerate(set(tag))}
ner_tags_numerical = [tag_to_ix[t] for t in tag]

ner_tags_onehot = torch.nn.functional.one_hot(torch.tensor(ner_tags_numerical))

print(ner_tags_onehot)
len(ner_tags_onehot)

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 

17

In [8]:
label_map ={}
j=0
for i in tag:
    # label_map[i] = ner_tags_onehot[j]
    label_map[i] = j
    j+=1
label_map

{'B-art': 0,
 'B-eve': 1,
 'B-geo': 2,
 'B-gpe': 3,
 'B-nat': 4,
 'B-org': 5,
 'B-per': 6,
 'B-tim': 7,
 'I-art': 8,
 'I-eve': 9,
 'I-geo': 10,
 'I-gpe': 11,
 'I-nat': 12,
 'I-org': 13,
 'I-per': 14,
 'I-tim': 15,
 'O': 16}

In [9]:
onehot_map ={}
j=0
for i in tag:
    # label_map[i] = ner_tags_onehot[j]
    onehot_map[label_map[i]] = ner_tags_onehot[j].tolist()
    j+=1
onehot_map

{0: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 1: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 2: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 3: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 4: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 5: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 6: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 7: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 8: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 9: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 10: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 11: [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 12: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 13: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 14: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 15: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 16: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [10]:
num_cls = len(label_map)
num_cls

17

In [11]:
y_train=[[label_map[word] for word in sentence] for sentence in train_label]

In [12]:
len(y_train), y_train[0]

(38367,
 [3,
  5,
  13,
  16,
  6,
  14,
  16,
  16,
  16,
  7,
  16,
  16,
  16,
  16,
  16,
  5,
  13,
  13,
  16,
  2,
  10,
  16])

In [13]:
y_val=[[label_map[word] for word in sentence] for sentence in val_label]
len(y_val),y_val[0]

(4796, [2, 16, 16, 16, 16, 16, 16, 5, 16, 16, 16, 16, 16, 16, 16, 16])

In [16]:
vocabulary = set(w for s in train_text for w in s )
print(len(vocabulary))

29078


In [17]:
# Add the unknown token to our vocabulary
vocabulary.add("<unk>")

In [18]:
# Add the <pad> token to our vocabulary
vocabulary.add("<pad>")

# Function that pads the given sentence
# We are introducing this function here as an example
# We will be utilizing it later in the tutorial
def pad_window(sentence, window_size, pad_token="<pad>"):
  window = [pad_token] * window_size
  return window + sentence + window

# Show padding example
window_size = 2
print(pad_window(train_text[0], window_size=window_size))

['<pad>', '<pad>', 'cuban', 'national', 'assembly', 'speaker', 'ricardo', 'alarcon', 'made', 'the', 'comment', 'friday', 'in', 'a', 'speech', 'before', 'the', 'un', 'world', 'summit', 'in', 'new', 'york', '.', '<pad>', '<pad>']


In [19]:
# We are just converting our vocabularly to a list to be able to index into it
# Sorting is not necessary, we sort to show an ordered word_to_ind dictionary
# That being said, we will see that having the index for the padding token
# be 0 is convenient as some PyTorch functions use it as a default value
# such as nn.utils.rnn.pad_sequence, which we will cover in a bit
ix_to_word = sorted(list(vocabulary))

# Creating a dictionary to find the index of a given word
word_to_ix = {word: ind for ind, word in enumerate(ix_to_word)}
len(word_to_ix)

29080

In [20]:
word_to_ix['<pad>']

1807

In [21]:
# Given a sentence of tokens, return the corresponding indices
def convert_token_to_indices(sentence, word_to_ix):
  indices = []
  for token in sentence:
    # Check if the token is in our vocabularly. If it is, get it's index.
    # If not, get the index for the unknown token.
    if token in word_to_ix:
      index = word_to_ix[token]
    else:
      index = word_to_ix["<unk>"]
    indices.append(index)
  return indices

# More compact version of the same function
def _convert_token_to_indices(sentence, word_to_ix):
  return [word_to_ind.get(token, word_to_ix["<unk>"]) for token in sentence]

# Show an example
example_sentence = ["we", "always", "come", "to", "kuwait"]
example_indices = convert_token_to_indices(example_sentence, word_to_ix)
restored_example = [ix_to_word[ind] for ind in example_indices]

print(f"Original sentence is: {example_sentence}")
print(f"Going from words to indices: {example_indices}")
print(f"Going from indices to words: {restored_example}")

Original sentence is: ['we', 'always', 'come', 'to', 'kuwait']
Going from words to indices: [28284, 2879, 6623, 26496, 15271]
Going from indices to words: ['we', 'always', 'come', 'to', 'kuwait']


In [22]:
# Converting our sentences to indices
example_padded_indices = [convert_token_to_indices(s, word_to_ix) for s in train_text]
len(example_padded_indices)

38367

In [23]:
# Creating an embedding table for our words
embedding_dim = 5
embeds = nn.Embedding(len(vocabulary), embedding_dim)

# Printing the parameters in our embedding table
list(embeds.parameters())

[Parameter containing:
 tensor([[ 0.6855, -1.2433,  0.6503,  1.7461, -1.5691],
         [-1.0888, -0.2956,  0.0582, -1.5052,  1.0458],
         [ 0.3358, -0.8030, -1.0766, -0.5173, -0.4558],
         ...,
         [ 0.0589,  1.7237,  1.7490, -1.0582, -0.8675],
         [ 0.4299, -0.4685, -1.3853,  0.8946,  0.4812],
         [ 1.3386, -0.0579,  1.6486, -1.9482, -1.3272]], requires_grad=True)]

In [24]:
# Get the embedding for the word Paris
index = word_to_ix["delhi"]
index_tensor = torch.tensor(index, dtype=torch.long)
paris_embed = embeds(index_tensor)
paris_embed

tensor([-0.3797,  0.6432, -1.1015, -0.4927,  2.3796],
       grad_fn=<EmbeddingBackward0>)

In [25]:
# We can also get multiple embeddings at once
index_paris = word_to_ix["delhi"]
index_ankara = word_to_ix["ankara"]
indices = [index_paris, index_ankara]
indices_tensor = torch.tensor(indices, dtype=torch.long)
embeddings = embeds(indices_tensor)
embeddings

tensor([[-0.3797,  0.6432, -1.1015, -0.4927,  2.3796],
        [-1.8665, -1.9956, -2.5430,  0.5626, -0.8039]],
       grad_fn=<EmbeddingBackward0>)

In [26]:
from torch.utils.data import DataLoader
from functools import partial

def custom_collate_fn(batch, window_size, word_to_ix):
  # Break our batch into the training examples (x) and labels (y)
  # We are turning our x and y into tensors because nn.utils.rnn.pad_sequence
  # method expects tensors. This is also useful since our model will be
  # expecting tensor inputs.
  x, y = zip(*batch)

  # Now we need to window pad our training examples. We have already defined a
  # function to handle window padding. We are including it here again so that
  # everything is in one place.
  def pad_window(sentence, window_size, pad_token="<pad>"):
    window = [pad_token] * window_size
    return window + sentence + window

  # Pad the train examples.
  x = [pad_window(s, window_size=window_size) for s in x]

  # Now we need to turn words in our training examples to indices. We are
  # copying the function defined earlier for the same reason as above.
  def convert_tokens_to_indices(sentence, word_to_ix):
    return [word_to_ix.get(token, word_to_ix["<unk>"]) for token in sentence]

  # Convert the train examples into indices.
  x = [convert_tokens_to_indices(s, word_to_ix) for s in x]

  # We will now pad the examples so that the lengths of all the example in
  # one batch are the same, making it possible to do matrix operations.
  # We set the batch_first parameter to True so that the returned matrix has
  # the batch as the first dimension.
  pad_token_ix = word_to_ix["<pad>"]

  # pad_sequence function expects the input to be a tensor, so we turn x into one
  x = [torch.LongTensor(x_i) for x_i in x]
  x_padded = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=pad_token_ix)

  # We will also pad the labels. Before doing so, we will record the number
  # of labels so that we know how many words existed in each example.
  lengths = [len(label) for label in y]
  lenghts = torch.LongTensor(lengths)

  y = [torch.LongTensor(y_i) for y_i in y]
  y_padded = nn.utils.rnn.pad_sequence(y, batch_first=True, padding_value=0)
  y_padded=torch.tensor([[onehot_map[int(word)] for word in sentence] for sentence in y_padded])
  # We are now ready to return our variables. The order we return our variables
  # here will match the order we read them in our training loop.
  return x_padded, y_padded, lenghts

In [27]:
def _custom_collate_fn(batch, window_size, word_to_ix):
  # Prepare the datapoints
  x, y = zip(*batch)
  x = [pad_window(s, window_size=window_size) for s in x]
  x = [convert_tokens_to_indices(s, word_to_ix) for s in x]

  # Pad x so that all the examples in the batch have the same size
  pad_token_ix = word_to_ix["<pad>"]
  x = [torch.LongTensor(x_i) for x_i in x]
  x_padded = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=pad_token_ix)

  # Pad y and record the length
  lengths = [len(label) for label in y]
  lenghts = torch.LongTensor(lengths)
  y = [torch.LongTensor(y_i) for y_i in y]
  y_padded = nn.utils.rnn.pad_sequence(y, batch_first=True, padding_value=0)
  y_padded=torch.tensor([[onehot_map[word] for word in sentence] for sentence in y_padded])
  return x_padded, y_padded, lenghts

In [28]:
# Parameters to be passed to the DataLoader
data = list(zip(train_text, y_train))
batch_size = 64
shuffle = True
window_size = 2
collate_fn = partial(custom_collate_fn, window_size=window_size, word_to_ix=word_to_ix)

# Instantiate the DataLoader
train_loader = DataLoader(data, batch_size=batch_size, shuffle=shuffle, collate_fn=collate_fn)

data = list(zip(val_text, y_val))
batch_size = 64
shuffle = True
window_size = 2
collate_fn = partial(custom_collate_fn, window_size=window_size, word_to_ix=word_to_ix)
# Instantiate the DataLoader
val_loader = DataLoader(data, batch_size=batch_size, shuffle=shuffle, collate_fn=collate_fn)
# Go through one loop
counter = 0
for batched_x, batched_y, batched_lengths in train_loader:
  print(f"Iteration {counter}")
  print("Batched Input:")
  print(batched_x)
  print("Batched Labels:")
  print(batched_y)
  print("Batched Lengths:")
  print(batched_lengths)
  print("")
  counter += 1
  break

Iteration 0
Batched Input:
tensor([[ 1807,  1807,  2998,  ...,  1807,  1807,  1807],
        [ 1807,  1807,  3008,  ...,  1807,  1807,  1807],
        [ 1807,  1807, 26220,  ...,  1807,  1807,  1807],
        ...,
        [ 1807,  1807, 26220,  ...,  1807,  1807,  1807],
        [ 1807,  1807, 28177,  ...,  1807,  1807,  1807],
        [ 1807,  1807, 10963,  ...,  1807,  1807,  1807]])
Batched Labels:
tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [1, 0, 0,  ..., 0, 0, 0],
         [1, 0, 0,  ..., 0, 0, 0],
         [1, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [1, 0, 0,  ..., 0, 0, 0],
         [1, 0, 0,  ..., 0, 0, 0],
         [1, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 1, 0],
         ...,
         [1, 0, 0,  ...,

In [29]:
def pad_sentence_for_window(sentence, window_size, pad_token="<pad>"):
    return [pad_token]*window_size + sentence + [pad_token]*window_size 

In [30]:
def convert_tokens_to_inds(sentence, word_2_id):
    return [word_2_id.get(t, word_2_id["<unk>"]) for t in sentence]

In [31]:
def my_collate(data, window_size, word_2_id):
    """
    For some chunk of sentences and labels
        -add winow padding
        -pad for lengths using pad_sequence
        -convert our labels to one-hots
        -return padded inputs, one-hot labels, and lengths
    """
    
    x_s, y_s = zip(*data)

    # deal with input sentences as we've seen
    window_padded = [convert_tokens_to_inds(pad_sentence_for_window(sentence, window_size), word_2_id)
                                                                                  for sentence in x_s]
    # append zeros to each list of token ids in batch so that they are all the same length
    padded = nn.utils.rnn.pad_sequence([torch.LongTensor(t) for t in window_padded], batch_first=True)
    
    # convert labels to one-hots
    labels = []
    lengths = []
    for y in y_s:
        lengths.append(len(y))
        label = torch.zeros((len(y),2 ))
        true = torch.LongTensor(y) 
        false = ~true.byte()
        label[:, 0] = false
        label[:, 1] = true
        labels.append(label)
    padded_labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)
    
    return padded.long(), padded_labels, torch.LongTensor(lengths)

# Model

In [33]:
class SoftmaxWordWindowClassifier(nn.Module):
    """
    A one-layer, binary word-window classifier.
    """
    def __init__(self, config, vocab_size, pad_idx=0):
        super(SoftmaxWordWindowClassifier, self).__init__()
        """
        Instance variables.
        """
        self.window_size = 2*config["half_window"]+1
        self.embed_dim = config["embed_dim"]
        self.hidden_dim = config["hidden_dim"]
        self.num_classes = config["num_classes"]
        self.freeze_embeddings = config["freeze_embeddings"]
        
        """
        Embedding layer
        -model holds an embedding for each layer in our vocab
        -sets aside a special index in the embedding matrix for padding vector (of zeros)
        -by default, embeddings are parameters (so gradients pass through them)
        """
        self.embed_layer = nn.Embedding(vocab_size, self.embed_dim, padding_idx=pad_idx)
        if self.freeze_embeddings:
            self.embed_layer.weight.requires_grad = False
        
        """
        Hidden layer
        -we want to map embedded word windows of dim (window_size+1)*self.embed_dim to a hidden layer.
        -nn.Sequential allows you to efficiently specify sequentially structured models
            -first the linear transformation is evoked on the embedded word windows
            -next the nonlinear transformation tanh is evoked.
        """
        self.hidden_layer = nn.Sequential(nn.Linear(self.window_size*self.embed_dim, 
                                                    self.hidden_dim), 
                                          nn.GELU())
        
        """
        Output layer
        -we want to map elements of the output layer (of size self.hidden dim) to a number of classes.
        """
        self.output_layer = nn.Linear(self.hidden_dim, self.num_classes)
        
        """
        Softmax
        -The final step of the softmax classifier: mapping final hidden layer to class scores.
        -pytorch has both logsoftmax and softmax functions (and many others)
        -since our loss is the negative LOG likelihood, we use logsoftmax
        -technically you can take the softmax, and take the log but PyTorch's implementation
         is optimized to avoid numerical underflow issues.
        """
        self.log_softmax = nn.LogSoftmax(dim=2)
        
    def forward(self, inputs):
        """
        Let B:= batch_size
            L:= window-padded sentence length
            D:= self.embed_dim
            S:= self.window_size
            H:= self.hidden_dim
            
        inputs: a (B, L) tensor of token indices
        """
        B, L = inputs.size()
        
        """
        Reshaping.
        Takes in a (B, L) LongTensor
        Outputs a (B, L~, S) LongTensor
        """
        # Fist, get our word windows for each word in our input.
        token_windows = inputs.unfold(1, self.window_size, 1)
        _, adjusted_length, _ = token_windows.size()
        
        # Good idea to do internal tensor-size sanity checks, at the least in comments!
        assert token_windows.size() == (B, adjusted_length, self.window_size)
        
        """
        Embedding.
        Takes in a torch.LongTensor of size (B, L~, S) 
        Outputs a (B, L~, S, D) FloatTensor.
        """
        embedded_windows = self.embed_layer(token_windows)
        
        """
        Reshaping.
        Takes in a (B, L~, S, D) FloatTensor.
        Resizes it into a (B, L~, S*D) FloatTensor.
        -1 argument "infers" what the last dimension should be based on leftover axes.
        """
        embedded_windows = embedded_windows.view(B, adjusted_length, -1)
        
        """
        Layer 1.
        Takes in a (B, L~, S*D) FloatTensor.
        Resizes it into a (B, L~, H) FloatTensor
        """
        layer_1 = self.hidden_layer(embedded_windows)
        
        """
        Layer 2
        Takes in a (B, L~, H) FloatTensor.
        Resizes it into a (B, L~, 2) FloatTensor.
        """
        output = self.output_layer(layer_1)
        
        """
        Softmax.
        Takes in a (B, L~, 2) FloatTensor of unnormalized class scores.
        Outputs a (B, L~, 2) FloatTensor of (log-)normalized class scores.
        """
        output = self.log_softmax(output)
        
        return output

# Training

In [34]:
def loss_function(outputs, labels, lengths):
    """Computes negative LL loss on a batch of model predictions."""
    B, L, num_classes = outputs.size()
    num_elems = lengths.sum().float()
        
    # get only the values with non-zero labels
    loss = outputs*labels
    
    # rescale average
    return -loss.sum() / num_elems

In [35]:
def train_epoch(loss_function, optimizer, model, train_data):
    
    ## For each batch, we must reset the gradients
    ## stored by the model.   
    total_loss = 0
    for batch, labels, lengths in train_data:
        # clear gradients
        optimizer.zero_grad()
        # evoke model in training mode on batch
        outputs = model.forward(batch)
        # compute loss w.r.t batch
        loss = loss_function(outputs, labels, lengths)
        # pass gradients back, startiing on loss value
        loss.backward()
        # update parameters
        optimizer.step()
        total_loss += loss.item()
    
    # return the total to keep track of how you did this time around
    return total_loss
    

In [214]:
def validate_epoch(loss_function, model, val_data):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch, labels, lengths in val_data:
            outputs = model.forward(batch)
            loss = loss_function(outputs, labels, lengths)
            total_loss += loss.item()
    return total_loss

In [224]:
config = {"batch_size": 64,
          "half_window": 2,
          "embed_dim": 25,
          "hidden_dim": 25,
          "num_classes": 17,
          "freeze_embeddings": False,
         }
learning_rate = .0002
num_epochs = 1000
model = SoftmaxWordWindowClassifier(config, len(word_to_ix))
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [225]:
from tqdm import tqdm

In [219]:
losses = []
for epoch in tqdm(range(num_epochs)):
    epoch_loss = train_epoch(loss_function, optimizer, model, train_loader)
    if epoch % 1 == 0:
        losses.append(epoch_loss)
print(losses)

100%|█████████████████████████████████████████████| 1/1 [00:08<00:00,  8.92s/it]

[2631.4190690517426]


# Training with validation

In [ ]:
# Training loop with early stopping
losses = []
val_losses = []
best_val_loss = float('inf')
patience_counter = 0
for epoch in tqdm(range(num_epochs)):
    epoch_loss = train_epoch(loss_function, optimizer, model, train_loader)
    val_loss = validate_epoch(loss_function, model, val_loader)
    if epoch % 1 == 0:
        losses.append(epoch_loss)
        val_losses.append(val_loss)
        print(f'Epoch {epoch}: Train Loss: {epoch_loss}, Validation Loss: {val_loss}')
        # Check for early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            # Save the best model
            torch.save(model.state_dict(), 'best_model.pt')
        else:
            patience_counter += 1
            if patience_counter >= 8:  # patience
                print("Early stopping due to validation loss not improving.")
                break

  0%|                                        | 1/1000 [00:09<2:46:11,  9.98s/it]

Epoch 0: Train Loss: 2399.1862387657166, Validation Loss: 242.26427030563354


  0%|                                        | 2/1000 [00:19<2:44:25,  9.89s/it]

Epoch 1: Train Loss: 1827.640622138977, Validation Loss: 214.08459424972534


  0%|                                        | 3/1000 [00:29<2:44:27,  9.90s/it]

...

Epoch 224: Train Loss: 430.9276941418648, Validation Loss: 53.560534715652466


 23%|████████▌                             | 226/1000 [37:19<2:15:05, 10.47s/it]

Epoch 225: Train Loss: 430.8782313466072, Validation Loss: 53.564945697784424


 23%|████████▋                             | 227/1000 [37:29<2:14:59, 10.48s/it]

Epoch 226: Train Loss: 430.6831773519516, Validation Loss: 53.53009009361267


# Prediction

In [223]:
y_pred=[]
y_true=[]
for test_instance, labs, _ in val_loader:
    outputs = model.forward(test_instance)
    # print(outputs[0])
    for o in outputs:
        fdir=torch.argmax(o,dim=1).tolist()
        for vy in fdir:
            y_pred.append(vy)
    for l in labs:
        fdir=torch.argmax(outputs[0], dim=1).tolist()
        for vx in fdir:
            y_true.append(vx)
    # print(test_instance)
    # print(len(outputs)," ",len(test_instance))
    # print(torch.argmax(outputs[0], dim=1))
    # print(labs[0])
    # print(torch.argmax(labs, dim=2))
print(y_true[0:5],"\n",y_pred[0:5])

[1, 1, 14, 1, 1, 12, 0, 12, 1, 16, 10, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] [1, 1, 14, 1, 1, 12, 0, 12, 1, 16, 10, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
from sklearn.metrics import f1_score
f1 = f1_score(y_true, y_pred, average='weighted')
print(f'F1 Score: {f1}')


# evaluating testdata

In [ ]:
dt = pd.read_csv('ner_val')
testdt = list(dt['sentence']).split()


In [ ]:
# Parameters to be passed to the DataLoader

data = list(zip(train_text, y_train))
batch_size = 64
shuffle = True
window_size = 2
collate_fn = partial(custom_collate_fn, window_size=window_size, word_to_ix=word_to_ix)

# Instantiate the DataLoader
train_loader = DataLoader(data, batch_size=batch_size, shuffle=shuffle, collate_fn=collate_fn)

data = list(zip(val_text, y_val))
batch_size = 64
shuffle = True
window_size = 2
collate_fn = partial(custom_collate_fn, window_size=window_size, word_to_ix=word_to_ix)
# Instantiate the DataLoader
val_loader = DataLoader(data, batch_size=batch_size, shuffle=shuffle, collate_fn=collate_fn)
# Go through one loop
counter = 0
for batched_x, batched_y, batched_lengths in train_loader:
  print(f"Iteration {counter}")
  print("Batched Input:")
  print(batched_x)
  print("Batched Labels:")
  print(batched_y)
  print("Batched Lengths:")
  print(batched_lengths)
  print("")
  counter += 1
  break